In [28]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
view_name='v_data_science_editor_pubmed_ids'
mview_name='mv_data_science_editor_pubmed_ids'
query_name ='v_data_science_editor_pubmed_ids.sql'

In [29]:
import logging

from bigquery_views_manager.update_views import update_or_create_view
from bigquery_views_manager.materialize_views import materialize_view

from data_science_pipeline.utils.bq import get_client
from data_science_pipeline.sql import get_sql

In [30]:
LOGGER = logging.getLogger(__name__)
logging.basicConfig(level='INFO')

In [31]:
client = get_client(project_id=project_id)

In [33]:
LOGGER.info('processing: %s', query_name)
view_query = get_sql(query_name).format(
        project=project_id,
        dataset=output_dataset)

LOGGER.info(view_query)

update_or_create_view(
        client=client,
        view_name=view_name,
        view_query=view_query,
        dataset=output_dataset
)

INFO:__main__:processing: v_data_science_editor_pubmed_ids.sql
INFO:__main__:WITH t_data_science_editor_pubmed_ids_with_rn AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY provenance.imported_timestamp DESC) as rn
    FROM `elife-data-pipeline.de_dev.data_science_editor_pubmed_ids` AS pubmed_ids
),

t_latest_data_science_editor_pubmed_ids AS (
    SELECT * FROM t_data_science_editor_pubmed_ids_with_rn
    WHERE rn = 1
)

SELECT * EXCEPT(rn)
FROM t_latest_data_science_editor_pubmed_ids

INFO:bigquery_views_manager.update_views:updated or replaced view: elife-data-pipeline:de_dev.v_data_science_editor_pubmed_ids


In [ ]:
materialize_view(
    client=client,
    source_view_name=view_name,
    destination_table_name=mview_name,
    project=project_id,
    source_dataset=source_dataset,
    destination_dataset=output_dataset
)